In [1]:
import pandas
import json
import requests
from time import sleep
from tqdm.notebook import tqdm, trange
import asyncio
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor
from timeit import default_timer
import multiprocessing
import nest_asyncio

In [2]:
game_names = open("data/game_names.txt", "r")

#some collissions (~300) happened, need to wait more on download
names_set = set()

for name in game_names:
    names_set.add(name)

In [3]:
def fetch(session, game_name, lock, file, pbar):
    base_url = "https://www.boardgamers.space/api/gameplay/"
    with session.get(base_url + game_name) as response:
        data = response.json()
        if response.status_code != 200:
            #print("FAILURE TO RETRTREIVE JSON::{0} {1}".format(url, response.error))
            return 1
        else:
            with lock:
                try:
                    file.write(json.dumps(data) + "\n")
                except Exception as err:
                    print(err)
                finally:
                    pbar.update(1)
                    return 0
    return 0

In [4]:
async def get_data_asynchronous(names_set, file_lock, file, pbar):
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        with requests.Session() as session:
            
            loop = asyncio.get_event_loop()
            
            tasks = [
                loop.run_in_executor(
                    executor,
                    fetch,
                    *(session, name, file_lock, file, pbar)
                )
                for name in names_set
            ]
            
            for response in await asyncio.gather(*tasks):
                pass

In [ ]:
pbar = tqdm(total=len(names_set))

nest_asyncio.apply()
m = multiprocessing.Manager()
lock = m.Lock()
loop = asyncio.get_event_loop()
with open("data/game_data_raw.txt", "a") as game_data_raw:
    future = asyncio.ensure_future(get_data_asynchronous(names_set, lock, game_data_raw, pbar))
    loop.run_until_complete(future)
    
    game_data_raw.close()

  0%|          | 0/35441 [00:00<?, ?it/s]